In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
# from __future__ import print_function

In [2]:
from rdkit import Chem

In [5]:
from harness import RMSE, train_test

ImportError: No module named harness

In [2]:
xtrain, xtest, ytrain, ytest=train_test()

Loading Training Data...
Splitting Data...


In [7]:
import numpy as np
np.shape(xtest)

(100000, 257)

In [8]:
xtest.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
276826,[nH]1cccc1-c1ccc(cn1)-c1sc(-c2nccs2)c2cc[nH]c12,0,0,0,0,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
849425,c1sc(-c2ccc(cc2)-c2sc(-c3cccc4c[nH]cc34)c3[se]...,0,0,0,0,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
504499,c1sc(-c2cc3[se]c4c(oc5ccc6cscc6c45)c3cn2)c2cc[...,0,0,0,0,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
601054,[nH]1c2C=C[SiH2]c2c2[nH]c3cc([se]c3c12)-c1cccc...,1,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
980221,[nH]1c(cc2c3cocc3c3c4ccccc4sc3c12)-c1cccc2cocc12,0,0,0,0,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [8]:
np.shape(b)

(100000, 258)

In [3]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [3]:
df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.19
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,1,0,0,1,0,0,0,0,0,1.60
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.49
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.36
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.98


In [4]:
df_test.head()

,Id,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,1,c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...,0,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
1,2,[nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1,0,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
2,3,[nH]1c2cc(-c3ccc[se]3)c3nsnc3c2c2c3cscc3c3ccc4...,1,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,4,[nH]1c(cc2cnc3c(c12)c1=C[SiH2]C=c1c1ccc2=CCC=c...,1,0,0,0,1,1,1,0,...,0,1,0,0,0,0,0,0,0,0
4,5,c1sc(-c2sc(-c3sc(-c4scc5[se]ccc45)c4ccoc34)c3c...,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [5]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [6]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1,0,0,0,1,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [7]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)


'\nExample Feature Engineering\n\nthis calculates the length of each smile string and adds a feature column with those lengths\nNote: this is NOT a good feature and will result in a lower score!\n'

In [8]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape

Train features: (1000000, 256)
Train gap: (1000000,)
Test features: (824230, 256)


In [9]:
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred = LR.predict(X_test)

In [10]:
# Prediction of LR on the training set
LR_train_pred = LR.predict(X_train)


In [11]:

rms = np.sqrt(mean_squared_error(Y_train, LR_train_pred))
rms

0.29892483586041568

In [12]:
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_pred = RF.predict(X_test)

In [13]:
# Prediction of RF on the training set
RF_train_pred = RF.predict(X_train)

In [14]:
rms = np.sqrt(mean_squared_error(Y_train, RF_train_pred))
rms

0.27187642510057108

In [ ]:
## In the provided code, the RF predictor performs better than LR on the training set, but the 
##     LR predictor performs better than RF on the test set.  


In [9]:
train_descrip = df_train.describe()
## Notice that some features have 0 values for all observations.  These columns contain no information, so perhaps
##     we should remove these from the model.  We could remove these columns from the training set, train the model, 
##     then remove the same columns from the test set before making the predictions.  
train_descrip

,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,feat_010,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
count,1000000.000000,1000000,1000000,1000000,1000000.000000,1000000.00000,1000000.000000,1000000,1000000,1000000,...,1000000,1000000.000000,1000000,1000000,1000000.000000,1000000.000000,1000000,1000000,1000000,1000000
mean,0.642013,0,0,0,0.999987,0.47765,0.975860,0,0,0,...,0,0.917407,0,0,0.218787,0.037309,0,0,0,0
std,0.479409,0,0,0,0.003606,0.49950,0.153484,0,0,0,...,0,0.275266,0,0,0.413424,0.189518,0,0,0,0
min,0.000000,0,0,0,0.000000,0.00000,0.000000,0,0,0,...,0,0.000000,0,0,0.000000,0.000000,0,0,0,0
25%,0.000000,0,0,0,1.000000,0.00000,1.000000,0,0,0,...,0,1.000000,0,0,0.000000,0.000000,0,0,0,0
50%,1.000000,0,0,0,1.000000,0.00000,1.000000,0,0,0,...,0,1.000000,0,0,0.000000,0.000000,0,0,0,0
75%,1.000000,0,0,0,1.000000,1.00000,1.000000,0,0,0,...,0,1.000000,0,0,0.000000,0.000000,0,0,0,0
max,1.000000,0,0,0,1.000000,1.00000,1.000000,0,0,0,...,0,1.000000,0,0,1.000000,1.000000,0,0,0,0


In [10]:
## In fact, 225 features have zero mean
sum(train_descrip.loc['mean']==0)

225

In [11]:
## But none have a mean of 1
sum(train_descrip.loc['mean']==1)

0

In [12]:
## There are 4 features that apply to less than 5% of the samples.  Maybe some of these would be worth removing 
##     as well. 
sum(train_descrip.loc['mean']>=(0.95))

4

In [32]:
np.array(train_descrip.loc['mean']==0)

array([False,  True,  True,  True, False, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True, False,  True,  True, False,
        True,  True,

In [37]:
## Names of all the features that are all zeroes in the training set
zeros = train_descrip.columns.values[np.array(train_descrip.loc['mean']==0)]

In [13]:
## To test ideas without using up submissions, it may be worth separating out a validation set from the training
##     data.  

In [14]:
df_train = pd.read_csv("train.csv")
from sklearn.cross_validation import train_test_split
df_train2, df_val = train_test_split(df_train, test_size = 0.1)

In [19]:
Y_train2 = df_train2['gap']
X_train2 = df_train2.drop(['gap', 'smiles'], axis=1)
Y_val = df_val['gap']
X_val = df_val.drop(['gap', 'smiles'], axis=1)

In [22]:
# First, fit a linear regression to the 90% training set, predict on the validation set, and measure RMSE
LR = LinearRegression()
LR.fit(X_train2, Y_train2)
LR_val_pred = LR.predict(X_val)

rms = np.sqrt(mean_squared_error(Y_val, LR_val_pred))
rms

0.29937849344738748

In [39]:
# Drop the columns that are filled with all zeroes
X_dropzeros = X_train2.drop(zeros, axis=1)
X_val_dropzeros = X_val.drop(zeros, axis=1)

In [40]:
# Fit a new linear regression on just these columns.  Note that the error is nearly identical.  This makes sense
#    because presumably the coefficients for those columns are zero, or the values for those columns in the 
#    validation set are still zero.  
LR = LinearRegression()
LR.fit(X_dropzeros, Y_train2)
LR_pred = LR.predict(X_val_dropzeros)
np.sqrt(mean_squared_error(Y_val, LR_pred))

0.29936871110915725

In [41]:
## Look at the regression coefficients.  Notice that the element in the 3rd and 28th position are exactly the opposite,
##    as are the 13th and 25th, and 22nd and 23rd.  And only 6 of these have values over 1.  3 are redundant, so there 
##    are only 3 real features here.  
LR.coef_

array([  2.21995998e-02,   1.83951555e-01,  -1.02568854e+10,
         2.19115552e-01,  -5.91795977e-02,   2.26147274e-01,
         3.27403202e-01,  -2.20970680e-01,  -2.88623728e-01,
        -1.78579087e-02,  -6.18655814e-02,  -9.42096446e-02,
        -8.30861900e+08,  -4.31399429e-01,  -3.03833937e-02,
         1.86457675e-02,  -1.29761743e-01,   3.91471041e-02,
        -2.17111479e-02,  -6.56619582e-02,  -3.11691090e-01,
        -1.56012319e+11,   1.56012319e+11,   1.89303058e-01,
         8.30861899e+08,  -1.38197638e-01,   1.41647705e-01,
         1.02568854e+10,  -7.15622892e-02,  -3.18174576e-01,
        -7.41296353e-02])

In [66]:
## These are the names of the columns that have LR coefficients less than one.  This includes the features that are
###    assigned low importance and the redundant features
dropcols = X_dropzeros.columns.values[LR.coef_ < 1]

In [67]:
X_dropzeros2 = X_dropzeros.drop(dropcols, axis=1)

In [71]:
X_dropzeros2.describe()

,feat_200,feat_218,feat_243
count,900000.000000,900000.000000,900000.000000
mean,0.000780,0.290554,0.477597
std,0.027918,0.454018,0.499498
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000


In [72]:
## Now we do the same column removal to the validation set
X_val_drop2 = X_val_dropzeros.drop(dropcols, axis=1)

LR = LinearRegression()
LR.fit(X_dropzeros2, Y_train2)
LR_pred = LR.predict(X_val_drop2)
np.sqrt(mean_squared_error(Y_val, LR_pred))
### This made the accuracy worse.  

0.37936499228023451

In [80]:
## Let's try PCA
pca = PCA(n_components = 8)
x2_pca_8 = pca.fit_transform(X_train2)
x2_pca_8

array([[-0.91673371,  0.28187405, -1.07566091, ...,  0.14134164,
        -0.02728798, -0.00363017],
       [ 1.45104241, -0.09515704, -0.26445954, ..., -0.40686422,
         0.37403813,  1.0846393 ],
       [ 1.33891262, -0.07029081, -0.25186749, ..., -0.13876578,
        -0.26095136, -0.7180509 ],
       ..., 
       [ 0.01752801, -1.78928217, -0.44841959, ...,  0.67633487,
         0.52940674,  0.12053991],
       [-1.87464971, -0.25581259,  0.74712058, ...,  0.91184441,
         0.47672613, -0.00915165],
       [-0.46945301,  1.06609136,  1.17539852, ...,  0.39415755,
         0.14194012, -0.13127874]])

In [82]:
xval_8 = pca.transform(X_val)

LR = LinearRegression()
LR.fit(x2_pca_8, Y_train2)
LR_pred = LR.predict(xval_8)
np.sqrt(mean_squared_error(Y_val, LR_pred))
## Shucks, that is worse than using all the data too.  

0.32116552168138951

In [84]:
for i in range(1,10):
    pca = PCA(n_components=i)
    x = pca.fit_transform(X_train2)
    xval_pca = pca.transform(X_val)
    LR = LinearRegression()
    LR.fit(x, Y_train2)
    LR_pred = LR.predict(xval_pca)
    print np.sqrt(mean_squared_error(Y_val, LR_pred))

0.401610355508
0.401161992945
0.378087742252
0.378017588982
0.347083319608
0.346921421014
0.322243971097
0.321165521681
0.313834208635


In [85]:
for i in range(10,20):
    pca = PCA(n_components=i)
    x = pca.fit_transform(X_train2)
    xval_pca = pca.transform(X_val)
    LR = LinearRegression()
    LR.fit(x, Y_train2)
    LR_pred = LR.predict(xval_pca)
    print np.sqrt(mean_squared_error(Y_val, LR_pred))

0.313826389401
0.313562856044
0.312958133546
0.312945736063
0.311477731581
0.30626911711
0.305969557171
0.303390862882
0.302883947172
0.302708932428


In [86]:
for i in range(20,30):
    pca = PCA(n_components=i)
    x = pca.fit_transform(X_train2)
    xval_pca = pca.transform(X_val)
    LR = LinearRegression()
    LR.fit(x, Y_train2)
    LR_pred = LR.predict(xval_pca)
    print np.sqrt(mean_squared_error(Y_val, LR_pred))

0.302692610904
0.302690760478
0.300786128641
0.300079266017
0.299743917947
0.29970089994
0.299380408172
0.299373001833
0.299372850773
0.299375775241


In [9]:
m = df_train['smiles'][0]

In [15]:
import sys
print (sys.path)


['', '/Users/christianjunge/OneDrive/CS181MachineLearning/Practical1', '/usr/local/lib/python2.7/site-packages', '/Users/christianjunge/anaconda/lib/python27.zip', '/Users/christianjunge/anaconda/lib/python2.7', '/Users/christianjunge/anaconda/lib/python2.7/plat-darwin', '/Users/christianjunge/anaconda/lib/python2.7/plat-mac', '/Users/christianjunge/anaconda/lib/python2.7/plat-mac/lib-scriptpackages', '/Users/christianjunge/anaconda/lib/python2.7/lib-tk', '/Users/christianjunge/anaconda/lib/python2.7/lib-old', '/Users/christianjunge/anaconda/lib/python2.7/lib-dynload', '/Users/christianjunge/.local/lib/python2.7/site-packages', '/Users/christianjunge/anaconda/lib/python2.7/site-packages/Sphinx-1.3.1-py2.7.egg', '/Users/christianjunge/anaconda/lib/python2.7/site-packages/setuptools-19.4-py2.7.egg', '/Users/christianjunge/anaconda/lib/python2.7/site-packages', '/Users/christianjunge/anaconda/lib/python2.7/site-packages/aeosa', '/Users/christianjunge/anaconda/lib/python2.7/site-packages/I

In [95]:
df_train['smiles'][300]

'c1sc(C2=Cc3c([SiH2]2)c2oc4ccccc4c2c2=C[SiH2]C=c32)c2[SiH2]C=Cc12'

In [12]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [48]:
write_to_file("sample1.csv", LR_pred)
write_to_file("sample2.csv", RF_pred)